In [166]:
import pandas as pd
import numpy as np

In [167]:
df = pd.read_csv("all_transactions.csv")
df.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock <div clas...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
1,10/31/2022,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
2,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock <div clas...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
3,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock <div clas...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
4,10/24/2022,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022


# Data Cleaning
1. remove html
2. ampersands
3. missing tickers

1. remove html

In [168]:
test = 'Microsoft Corporation - Common Stock <div class="text-muted">Option Type: Short Sale <br><em>Strike price:</em> $245.00 <br> <em>Expires:</em> 06/16/2023 </div>'
test

'Microsoft Corporation - Common Stock <div class="text-muted">Option Type: Short Sale <br><em>Strike price:</em> $245.00 <br> <em>Expires:</em> 06/16/2023 </div>'

In [169]:
inDiv = df['asset_description'].str.contains('<div class="text-muted">')
nonMuted = df['asset_description'].str.contains('<div')
df[ ~ inDiv &  nonMuted ]

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date


In [170]:
def removeHTML(descString):
    removeStrings=['<br>', 
    '<em>',
    '</em>', 
    '</div>', 
    '<div class="text-muted">'
    ]
    rez = descString
    for s in removeStrings:
        rez = rez.replace(s, '')

    return rez

removeHTML(test)



'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 '

In [171]:
df['asset_description'] = df['asset_description'].apply(lambda row: removeHTML(row))
df.head()


,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
1,10/31/2022,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
2,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
3,10/28/2022,Joint,MSFT,Microsoft Corporation - Common Stock Option Ty...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022
4,10/24/2022,Joint,MSFT,Microsoft Corporation - Common Stock,Stock,Purchase,"$15,001 - $50,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/b...,11/10/2022


In [172]:
df['id'] = df.index
# df.head()

2. ampersands

In [173]:
garbageString = 'amp;'
for col in df.columns: 

    currentFrame = df[df[col].astype(str).str.contains(garbageString)]
    affectedRows = currentFrame.shape[0]
    if affectedRows>0:
        print(col, affectedRows)


asset_description 589
comment 15


In [174]:
affectedColumns = ['asset_description','comment']

for col in affectedColumns:
    df[col] = df[col].str.replace(garbageString, '')

df[df['asset_description'].str.contains(garbageString)].shape[0]

0

3. missing tickers

In [175]:

df.isnull().sum()

transaction_date       0
owner                467
ticker               467
asset_description      0
asset_type           666
type                 467
amount                 0
comment              467
senator                0
ptr_link               0
disclosure_date        0
id                     0
dtype: int64

In [176]:
df[(df['owner'].isnull()) & ~(df['type'].isnull()) ].shape[0]



0

In [177]:
fillString = 'Unknown'
for col in ['owner', 'ticker','type']:
    df[col] = df[col].fillna(fillString)

df.owner.isnull().sum()

0

In [178]:
# df[df['asset_type'].isnull()].sample(n=10)
# df[df['asset_type'].isnull()].type.value_counts()

df['asset_type'] = df['asset_type'].fillna(fillString)
df.asset_type.isnull().sum()

0

# Load to DB


In [179]:
def trimText(row, maxText=249):
    row = str(row)
    row = row[:maxText]
    return row

df['comment'] = df['comment'].apply(lambda row: trimText(row))

In [180]:
def findMaxChars(col):
    maxLen =0
    listCol = df[col].to_list()
    for row in listCol:
        lenRow = len(str(row))
        if lenRow > maxLen:
            maxLen = lenRow

    return maxLen 
    
for col in df.columns:
    print(col, findMaxChars(col))

transaction_date 10
owner 7
ticker 18
asset_description 241
asset_type 28
type 14
amount 25
comment 249
senator 26
ptr_link 84
disclosure_date 10
id 4


In [181]:
import psycopg2 
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os


load_dotenv()


hostname=os.getenv("hostname")
database=os.getenv("database")
username=os.getenv("username")
pwd=os.getenv("pwd")
port_id=os.getenv("port_id")
host = os.getenv("host")




Python-dotenv could not parse statement starting at line 12


In [182]:


conn=None
cur=None 
db = None

def postDataFrameDB():
    try:
        conn_string = f"postgresql://{username}:{pwd}@{host}:{port_id}/{database}"
        db =  create_engine(url=conn_string)
        conn=db.connect()

        df.to_sql('transactions', con=conn, if_exists='replace', index=False)

        connPsy = psycopg2.connect(conn_string)
        cur = connPsy.cursor()
        connPsy.autocommit = True

        sqlString = """
        select * from transactions limit 5;
        """
        cur.execute(sqlString)

        records = cur.fetchall()
        for row in records:
            print(row)

    except Exception as err:
        print(err)
    finally: 
        if cur is not None: 
            cur.close()
        if conn is not None: 
            conn.close()
    
postDataFrameDB()


('10/28/2022', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '11/10/2022', 0)
('10/31/2022', 'Joint', 'CLF', 'Cleveland-Cliffs Inc. Common Stock', 'Stock', 'Purchase', '$15,001 - $50,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '11/10/2022', 1)
('10/28/2022', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $245.00  Expires: 06/16/2023 ', 'Stock Option', 'Sale (Full)', '$1,001 - $15,000', '--', 'Thomas H Tuberville', 'https://efdsearch.senate.gov/search/view/ptr/bdd81552-894d-4dc5-b1f3-9fe349529bf8/', '11/10/2022', 2)
('10/28/2022', 'Joint', 'MSFT', 'Microsoft Corporation - Common Stock Option Type: Short Sale Strike price: $24